In [161]:
import tensorflow as tf
import pandas as pd
import numpy as np
import string
import re
import random as rand

In [9]:
press = pd.read_csv('./data/press.csv')[['newspaper', 'text']]
press

,newspaper,text
0,20minutes,CORONAVIRUS Après une alerte lancée il y a tro...
1,bfm,"Parmi les trois foyers identifiés, l'un se tro..."
2,cnews,La France a enregistré 16 décès supplémentaire...
3,lacroix,Ne pas baisser la garde. C’est en résumé le me...
4,ledauphine,Un cas positif de Covid-19 a été détecté ce ve...
5,lefigaro,"Sur un marché parisien, les commerçants prenne..."
6,lemonde,"Le Brésil comptait, samedi 20 juin, près de 1 ..."
7,lesechos,Par Les Echos\n\nPublié le 20 juin 2020 à 18h4...


In [52]:
def clean_string2(text):
    clean_text = text.lower()
    clean_text = re.sub('covid-19', 'coronavirus', clean_text)
    clean_text = re.sub('[#:/\%€@»«➡️—-]', '', clean_text)
    clean_text = re.sub('’', ' \' ', clean_text)
    clean_text = re.sub('[éèëê]', 'e', clean_text)
    clean_text = re.sub('[àâ]', 'a', clean_text)
    clean_text = re.sub('[ïî]', 'i', clean_text)
    clean_text = re.sub('[öô]', 'o', clean_text)
    clean_text = re.sub("['\n\t]", ' ', clean_text)
    
    clean_text = re.sub(f"([{string.punctuation}])", r" \1 ", clean_text)
    clean_text = re.sub('\w*\d\w*', '<NB>', clean_text)  # Remove numbers
    return ' '.join(clean_text.split())


press['prepared_text'] = press.apply(lambda r: clean_string2(r['text']), axis=1)
press.iloc[0]['prepared_text'][:1000]

'coronavirus apres une alerte lancee il y a trois semaines sur un foyer de contaminations chez les saisonniers des bouchesdurhone , les nouveaux cas diminuent des travailleurs agricoles attendent pour se faire depister dans l ouest des bouchesdurhone . clement mahoudeau afp un foyer de contamination au coronavirus chez les travailleurs saisonniers de l ouest des bouchesdurhone a ete detecte au debut du mois de juin . les services de l etat ont eu du mal pour placer en quarantaine ces travailleurs saisonniers a cause de leurs conditions d hebergements souvent compliquees . sur <NB> travailleurs testes positifs , <NB> sont sortis de leur quarantaine . la bataille n est pas encore gagnee . le prefet des bouchesdurhone , pierre dartout , et le directeur de l agence regionale de sante de provence alpes cote d azur , philippe de mester , ont fait le point sur le foyer de contaminations au coronavirus chez des travailleurs saisonniers de l ouest du departement des bouchesdurhone . les nouvell

In [53]:
# Focus on one newspaper
newspaper = 'cnews'
assert newspaper in press.newspaper.values

based_text = press.set_index('newspaper').loc[newspaper, 'prepared_text']
print(len(based_text.split(' ')))
based_text[:1000]

4741


'la france a enregistre <NB> deces supplementaires lies au coronavirus dans les dernieres <NB> heures , portant le total des morts a <NB> . <NB> depuis le debut de l epidemie , selon le bilan publie samedi par sante publique france . selon l agence nationale de sante publique , le nombre de patients du coronavirus en reanimation continue parallelement de baisser avec <NB> malades , soit <NB> de moins que dans le bilan publie la veille par la direction generale de la sante ( dgs ) . au total , <NB> . <NB> personnes sont mortes dans les hopitaux , selon sante publique france . les donnees des deces en etablissements sociaux et medicosociaux , dont les ehpad , seront quant a elles actualisees mardi . le nombre total des personnes hospitalisees continue aussi de baisser , avec desormais <NB> . <NB> patients dans tout le pays . deux departements continuent d etre en situation de vulnerabilite elevee la guyane et mayotte . dans tout le territoire , le nombre de foyers epidemiques ( clusters 

In [58]:
# Define vocabulary
vocab = sorted(set(based_text.split(' ')))

In [59]:
# Define mappers
word2idx = {word: i for i, word in enumerate(vocab)}
idx2word = {i: word for i, word in enumerate(vocab)}

In [86]:
# Define a Dataset of sequences with a specific size

chunk_size = 11
chunks = []
whole_sequence = based_text.split(' ')
for i in range(len(whole_sequence) - chunk_size):
    chunk = [word2idx[whole_sequence[i+t]] for t in range(chunk_size) if i+t < len(whole_sequence)]
    chunks.append(chunk)

In [87]:
# Define targets
y = []
for chunk in chunks:
    target = chunk.pop(-1)
    y.append(target)

In [88]:
X = np.array(chunks)
y = np.array(y)

In [148]:
# Create model
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(len(vocab)+1, 100))
    model.add(tf.keras.layers.SimpleRNN(128))
    model.add(tf.keras.layers.Dense(len(vocab), activation='softmax'))
    model.compile(optimizer='adam', loss='')
    return model

In [158]:
model = get_model()

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=False)

model.compile(optimizer='adam', loss=loss)

In [159]:
model.fit(X, y, epochs=50, shuffle=True)

Epoch 1/50
148/148 [==============================] - 1s 7ms/step - loss: 6.1430
Epoch 2/50
148/148 [==============================] - 1s 6ms/step - loss: 5.6111
Epoch 3/50
148/148 [==============================] - 1s 6ms/step - loss: 5.2658
Epoch 4/50
148/148 [==============================] - 1s 6ms/step - loss: 4.8713
Epoch 5/50
148/148 [==============================] - 1s 6ms/step - loss: 4.5041
Epoch 6/50
148/148 [==============================] - 1s 6ms/step - loss: 4.1396
Epoch 7/50
148/148 [==============================] - 1s 6ms/step - loss: 3.7851
Epoch 8/50
148/148 [==============================] - 1s 6ms/step - loss: 3.4319
Epoch 9/50
148/148 [==============================] - 1s 6ms/step - loss: 3.0833
Epoch 10/50
148/148 [==============================] - 1s 7ms/step - loss: 2.7415
Epoch 11/50
148/148 [==============================] - 1s 7ms/step - loss: 2.4068
Epoch 12/50
148/148 [==============================] - 1s 6ms/step - loss: 2.1002
Epoch 13/50
148/148 [====

In [180]:
def generate_text(model, start_string):
    num_generate_tokens = 200
    
    # Encode start string
    clean_start = clean_string2(start_string)
    start_sequence = np.array([word2idx[word] for word in clean_start.split(' ')]).reshape(1, chunk_size-1)
    
    text_generated = clean_start.split(' ')
    model.reset_states()
    current_sequence = start_sequence
    for i in range(num_generate_tokens):
        predictions = model(current_sequence)
        
        # Add randomness
        p = rand.choice([1, 1, 1, 1, 1])
        next_idx = rand.choice(np.argsort(predictions[0])[-p:][::-1])
        next_word = idx2word[next_idx]
        text_generated.append(next_word)
        
        # Copy n-1 last words
        current_sequence = np.concatenate([current_sequence[0][1:], [next_idx]], axis=0).reshape(1, chunk_size-1)
        
    # Clean generated text
    final_text = ''
    for i in range(len(text_generated)):
        
        if i != 0 and text_generated[i-1] == '.' and text_generated[i] not in string.punctuation and text_generated[i] != '<NB>':
            clean = text_generated[i][0].upper() + text_generated[i][1:]
            clean += ' '
        elif i < len(text_generated)-1 and text_generated[i+1] in ('.', ',', '\''):
            clean = text_generated[i]
        else:
            clean = text_generated[i]
            clean += ' '
        final_text += clean
        
    return final_text


start_string = 'De nouveaux cas sont apparus en France. Le gouvernement'
generate_text(model, start_string)

'de nouveaux cas sont apparus en france. Le gouvernement a annonce. La nouvelle et de capacites en etablissements n a premier sanitaire sur le coronavirus ministre edouard philippe a que les croisieres suscite pour certaines libertes de empathie et. Les etatsunis, ainsi sont le fait ministre d ailleurs lundi a une strategie que conseil de defense du <NB>, qu ils ailleurs l actualite sur le coronavirus ici depuis le debut de l epidemie. Celleci est encore loin de reculer ailleurs dans le monde. Le maroc a fait etat d une hausse record du nombre de contaminations avec <NB> nouveaux cas, soit le bilan quotidien le plus eleve dans le royaume depuis l annonce du premier cas en mars. Ce qui n a pas empeche le gouvernement marocain d annoncer vendredi un nouvel assouplissement des restrictions en vigueur depuis la mimars. Inquietudes de l oms et c est bien ce qui inquiete l organisation mondiale de la sante. Le monde est entre dans une phase dangereuse, a mis en garde vendredi l oms. Le virus